In [1]:
import cfbd
import json
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import json
import os 

configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'j5Z7/ASYjq4l6PMJFJpY7WxwO5BtbY/U2TFBOVdziGKRvjI8WajGUnVRiUBGrIX8'
configuration.api_key_prefix['Authorization'] = 'Bearer'

games_api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
metrics_api_instance = cfbd.MetricsApi(cfbd.ApiClient(configuration))
plays_api_instance = cfbd.PlaysApi(cfbd.ApiClient(configuration))
drives_api_instance = cfbd.DrivesApi(cfbd.ApiClient(configuration))

In [2]:
def clean_teamname(team):
    # Special cases
    if team == 'olemiss':
        return 'ole miss'
    if team == 'sanjosestate':
        return 'san josé state'
    if team == 'usf':
        return 'south florida'
    
    # Ending token (georgia tech)     
    if team[-5:] == 'state':
        team = team[:-5] + ' state'
    if team[-4:] == 'tech':
        team = team[:-4] + ' tech'
    if team[-2:] == 'am':
        team = team[:-2] + ' a&m'
    if team[-4:] == 'dame':
        team = team[:-4] + ' dame'
    if team[-7:] == 'college':
        team = team[:-7] + ' college'
    
    # Beginning token (north dakota)     
    if team[:5] == 'north':
        team = 'north ' + team[5:]
    if team[:5] == 'south':
        team = 'south ' + team[5:]
    if team[:7] == 'central':
        team = 'central ' + team[7:]
    if team[:4] == 'west':
        team = 'west ' + team[4:]
    return team

In [3]:
# Find top 100 files by line length
# From filename, get team 1 and team 2
# From random comment, get a timestamp --> week number

name_of_dir = 'threads'
  
# Storing list of all files 
# in the given directory in list_of_files 
list_of_files = filter( lambda x: os.path.isfile 
                       (os.path.join(name_of_dir, x)), 
                        os.listdir(name_of_dir) ) 
  
# Sort list of file names by size  
list_of_files = sorted( list_of_files, 
                        key =  lambda x: os.stat 
                       (os.path.join(name_of_dir, x)).st_size)

list_of_files.reverse()
# list_of_files = list_of_files[99:102]
# print(len(list_of_files))
# print(list_of_files)

team1 = None
team2 = None
matchups = []

for filename in list_of_files:
    teams = filename.split("_")
    team1 = clean_teamname(teams[0])
    team2 = clean_teamname(teams[1][:-4])
    matchups.append((team1, team2, filename[:-4]))
# print(matchups)

In [ ]:
YEAR = 2023

# TODO: timezones
for team1, team2, filename in matchups:
    
    if os.path.exists(f'drive_win_probs/{filename}.json'):
        continue
    
    games = None
    season_type = 'regular'
    # Special case: oregon, washington played twice     
    if team1=='oregon' and team2=='washington':
        # PAC12 Championship game
        games = games_api_instance.get_games(year=YEAR, home=team2, away=team1, week=14, season_type=season_type)
    elif team1=='washington' and team2=='oregon':
        # Regular season game at washington
        games = games_api_instance.get_games(year=YEAR, home=team1, away=team2, week=7, season_type=season_type)
    else:
        games = games_api_instance.get_games(year=YEAR, home=team1, away=team2)
        if len(games) == 0:
            games = games_api_instance.get_games(year=YEAR, home=team2, away=team1)
        if len(games) == 0:
            season_type = 'postseason'
            games = games_api_instance.get_games(year=YEAR, home=team1, away=team2, season_type=season_type)
        if len(games) == 0:
            season_type='postseason'
            games = games_api_instance.get_games(year=YEAR, home=team2, away=team1, season_type=season_type)
    if len(games)==0:
        print('no games found')
        print(team1, team2)

    game = games[0]
    output = []
    print(f'id: {game._id} home: {game._home_team} away: {game._away_team}')
    wp = metrics_api_instance.get_win_probability_data(game_id=game._id)
    drives1 = drives_api_instance.get_drives(year=YEAR, season_type=season_type, week=game._week, offense=game._home_team, defense=game._away_team)
    drives2 = drives_api_instance.get_drives(year=YEAR, season_type=season_type, week=game._week, defense=game._home_team, offense=game._away_team)
    drives = []
    i=0
    j=0
    while i < len(drives1) and j < len(drives2):
        if drives1[i]._drive_number < drives2[j]._drive_number:
            drives.append(drives1[i])
            i += 1
        else:
            drives.append(drives2[j])
            j += 1
    while i < len(drives1):
        drives.append(drives1[i])
        i += 1
    while j < len(drives2):
        drives.append(drives2[j])
        j += 1
    
    plays1 = plays_api_instance.get_plays(year=YEAR, week=game._week, offense=game._home_team, defense=game._away_team, season_type=season_type)
    plays2 = plays_api_instance.get_plays(year=YEAR, week=game._week, offense=game._away_team, defense=game._home_team, season_type=season_type)
    plays = []
    i=0
    j=0
    while i < len(plays1) and j < len(plays2):
        if plays1[i]._drive_number < plays2[j]._drive_number:
            plays.append(plays1[i])
            i += 1
        elif plays1[i]._drive_number == plays2[j]._drive_number and plays1[i]._play_number < plays2[j]._play_number:
            plays.append(plays1[i])
            i += 1
        else:
            plays.append(plays2[j])
            j += 1
    while i < len(plays1):
        plays.append(plays1[i])
        i += 1
    while j < len(plays2):
        plays.append(plays2[j])
        j += 1
    # print(plays)
    
    # plays = plays1+plays2
    # if len(drives) == 0:
    #     drives = drives_api_instance.get_drives(year=YEAR, season_type='postseason', week=game._week, team=game._home_team)
    # print(drives)
    
    # metrics_api_instance.get_win_probability_data(game_id=game._id)
    if not wp:
        print('no wp')
        print(team1, team2)
        continue
    print(f'initial win prob: {wp[0]._home_win_prob:.3f}, final: {wp[-1]._home_win_prob:.3f}')
    curr_drive = 0
    for i in range(len(wp)):
        j = i
        while (j < len(plays) and wp[i]._play_id != plays[j]._id):
            # print(wp[i]._play_id)
            # print(plays[j]._id)
            j += 1

        dt_utc = None
        if j == len(plays) or plays[j]._wallclock is not None:
            if j == len(plays) or ():
                dt_utc = output[-1]['start_utc'] + 1
            else:
                dt_utc = datetime.fromisoformat(plays[j]._wallclock).astimezone(tz=None).timestamp()
            # Don't allow timestamp for a given play to be before a previous play!
            if i > 1 and dt_utc < output[-1]['start_utc']:
                dt_utc = output[-1]['start_utc'] + 1
            if (j == len(plays) or wp[i]._play_id == plays[j]._id):
                # print(plays[j])
                if j == len(plays) or plays[j].drive_number != curr_drive:
                    start_wp = wp[0]._home_win_prob
                    start_utc = int(datetime.fromisoformat(plays[0]._wallclock).astimezone(tz=None).timestamp())
                    # if game._home_team ==game._away_team == 'Washington':
                    # print(output)
                    # # print(game._week)
                    # print(j)
                    # print(np.shape(drives))
                    # print(drives)
                    delta_wp = 0
                    if len(output):
                        start_wp = output[-1]['end_win_prob']
                        start_utc = output[-1]['end_utc']
                        delta_wp = wp[i]._home_win_prob - start_wp
                    
                    # print(curr_drive)
                    agg_play = {
                        'drive_number' : curr_drive,
                        'drive_result' : drives[curr_drive-1]._drive_result,
                        'start_win_prob' : start_wp,
                        'end_win_prob' : wp[i]._home_win_prob,
                        'delta_win_prob': delta_wp,
                        'start_utc': start_utc,
                        'end_utc': int(dt_utc)
                    }
                    if j != len(plays):
                        curr_drive = plays[j].drive_number
                    output.append(agg_play)
                    # print(agg_play)
    output.pop(0)
    # print(output)
    with open(f'drive_win_probs/{filename}.json', 'w') as f:
        json.dump(output, f)
        # Plotting win percentage
#     time = [play['utc'] for play in output]
#     wp_list = [play['home_win_prob'] for play in output]
#     plot = plt.plot(time, wp_list)
#     plt.xlabel('UTC time')
#     plt.ylabel('Home Win Prob')
#     plt.title(f'(H) {game._home_team} vs (A) {game._away_team}')
#     plt.show()

#     plot = plt.plot(range(len(output)), wp_list)
#     plt.xlabel('Play Number')
#     plt.ylabel('Home Win Prob')
#     plt.title(f'(H) {game._home_team} vs (A) {game._away_team}')
#     plt.show()
#     print(game._week, game._season_type)
#     break

Skipping colorado_coloradostate
Skipping michigan_ohiostate
Skipping alabama_michigan
Skipping washington_michigan
Skipping texas_washington
Skipping lsu_floridastate
Skipping ohiostate_pennstate
Skipping colorado_nebraska
Skipping duke_clemson
Skipping oregon_washington
Skipping alabama_texas
Skipping tcu_colorado
Skipping georgia_alabama
Skipping oregon_colorado
Skipping notredame_ohiostate
Skipping pennstate_michigan
id: 401539478 home: Florida State away: Louisville
initial win prob: 0.448, final: 1.000
Skipping oklahoma_texas
Skipping georgia_floridastate
Skipping utah_florida
Skipping usc_arizona
Skipping missouri_ohiostate
Skipping michigan_iowa
Skipping washington_oregon
id: 401539479 home: Texas away: Oklahoma State
initial win prob: 0.453, final: 1.000
Skipping notredame_usc
Skipping clemson_floridastate
Skipping auburn_alabama
id: 401525874 home: Kansas away: Oklahoma
initial win prob: 0.316, final: 1.000
Skipping colorado_usc
Skipping navy_notredame
Skipping minnesota_nebra